In [6]:
import pandas as pd
import requests as rq

In [24]:
all_municipalities = pd.read_excel('Data/be-b-00.04-agv-20050313.xlsx', sheet_name='Gemeindeliste-Liste d. communes')['GDENAME'].tolist()
given_municipalities = pd.read_csv('Data/municipality_to_latitude_longitude.csv')['Municipality'].tolist()
to_check = set(all_municipalities) - set(given_municipalities)
to_check = list(to_check)

In [7]:
new_mun_coordinates = []
problems = []
len_todo = len(to_check)
for i, municipality in enumerate(to_check):
    api_url = f'https://api.api-ninjas.com/v1/geocoding?city={municipality}&country=Switzerland'
    response = rq.get(api_url + municipality, headers={'X-Api-Key': '8AmHgsXCnbvi7hlJ0Mh/hQ==rt399afLziw2LIfI'})
    if response.status_code == rq.codes.ok:
        if len(response.json())== 0:
            problems.append(municipality)
            print("Error with ", municipality)
            continue
        name = response.json()[0].get("name")
        latitude = response.json()[0].get("latitude")
        longitude = response.json()[0].get("longitude")
        new_mun_coordinates.append([name, latitude, longitude])
    else:
        print("Error:", response.status_code, response.text)
        problems.append(municipality)
    if i %50== 0:
        print(i, " of ", len_todo, " done")

columns = ['Municipality', 'Latitude', 'Longitude']
df_new_mun_coordinates = pd.DataFrame.from_records(new_mun_coordinates, columns=columns)
df_new_mun_coordinates.to_csv("Data/batch4_municipality_coordinates_in_progress")
df_problems = pd.DataFrame(problems)
df_problems.to_csv('Data/final_problems.csv')

0  of  232  done
Error with  Riom-Parsonz
Error with  Oberhofen (AG)
Error with  Tinizong-Rona
Error with  Obererlinsbach
50  of  232  done
Error with  St. Antönien Ascharina
Error with  Nesslau-Krummenau
Error with  Klosters-Serneus
Error with  Suraua
Error with  Schwanden (GL)
Error with  Villaz-Saint-Pierre
Error with  Niedererlinsbach
Error with  Rohr (AG)
Error with  Unterbözberg
Error with  Casti-Wergenstein
Error with  Reckingen-Gluringen
Error with  Tüscherz-Alfermée
Error with  Gettnau
Error with  Thielle-Wavre
Error with  Willisau Stadt
Error with  Bibern (SO)
Error with  Wiesen (GR)
Error with  Willisau Land
Error with  Isorno
Error with  Rüti (GL)
200  of  232  done
Error with  Rapperswil (SG)
Error with  Münster-Geschinen
Error with  Heinrichswil-Winistorf
Error with  Niederried bei Kallnach


In [11]:
df_batch4 = pd.read_csv('Data/batch4_municipality_coordinates_in_progress')
df_all_mun = pd.read_csv('Data/municipality_to_latitude_longitude.csv')
df_all = pd.concat([df_batch4, df_all_mun])

In [15]:
df_all = df_all[['Municipality', 'Latitude', 'Longitude']]
df_all[df_all.duplicated()]

,Municipality,Latitude,Longitude
180,Hoffen,48.925900,7.942913
14,Adlikon b. Andelfingen,47.582503,8.691256
19,Sauris,46.465366,12.708717
33,Trullikon,47.637726,8.693416
58,Wil,47.605521,8.504906
...,...,...,...
2623,Wohlen,47.350495,8.278691
2655,Puri,19.807608,85.825254
2656,Grone,45.727504,9.911202
2667,Étueffont,47.723257,6.922824


In [17]:
df_all = df_all.drop_duplicates()

In [18]:
df_all

,Municipality,Latitude,Longitude
0,Veurne,51.070957,2.654850
1,Sales,45.874277,5.959858
2,Erlinsbach (AG),47.396866,8.013374
3,Schinznach,47.446799,8.142284
4,Eschenbach,47.133023,8.317969
...,...,...,...
2723,Küttigen,47.419454,8.043257
2724,Ursins,46.734894,6.668296
2725,Laconnex,46.155596,6.036031
2726,Chavornay,45.880811,5.728952


In [28]:
df_all_mun_from_file = df_all['Municipality'].tolist()
differences = list(set(all_municipalities).difference(set(df_all_mun_from_file)))
differences

['Furna',
 'Sâles',
 'Erlinsbach',
 'Schinznach-Dorf',
 'Eschenbach (LU)',
 'Ecublens (VD)',
 'Riom-Parsonz',
 'Hauterive (NE)',
 'Vals',
 'Wil (AG)',
 'Brienz (BE)',
 'Veltheim (AG)',
 'Oberdorf (BL)',
 'Altdorf (SH)',
 'Bibern (SH)',
 'Hauterive (FR)',
 'Thalheim (AG)',
 'La Punt-Chamues-ch',
 'Stein (AG)',
 'Langnau bei Reiden',
 'Oberhofen (AG)',
 'Vandoeuvres',
 'Astano',
 'Zürich',
 'Neudorf',
 'Oberwil (BL)',
 'Eichberg',
 'Arbaz',
 'Saas Almagell',
 'Tinizong-Rona',
 'Waldenburg',
 'Obersaxen',
 'Laufenburg',
 'Marbach (SG)',
 'Neuheim',
 'Castaneda',
 'Wahlen',
 'Matten bei Interlaken',
 'Feldis/Veulden',
 'Crans-près-Céligny',
 'Bühl',
 'Mon',
 'Fribourg',
 'Dorf',
 'Montet (Glâne)',
 'Obererlinsbach',
 'Pfeffikon',
 'Niederwil (SO)',
 'Sulz (AG)',
 'Hofen',
 'Forel (FR)',
 'Blatten',
 'Saint-Ursanne',
 'Reichenburg',
 'Luzern',
 'Sulz (LU)',
 'St. Antönien Ascharina',
 'Sent',
 'Erlen',
 'Rekingen (AG)',
 'Safien',
 'Belmont-sur-Lausanne',
 'Nesslau-Krummenau',
 'Reiden',
 '

In [29]:
new_mun_coordinates = []
problems = []
len_todo = len(to_check)
for i, municipality in enumerate(differences):
    api_url = f'https://api.api-ninjas.com/v1/geocoding?city={municipality}&country=Switzerland'
    response = rq.get(api_url + municipality, headers={'X-Api-Key': '8AmHgsXCnbvi7hlJ0Mh/hQ==rt399afLziw2LIfI'})
    if response.status_code == rq.codes.ok:
        if len(response.json())== 0:
            problems.append(municipality)
            print("Error with ", municipality)
            continue
        name = municipality
        latitude = response.json()[0].get("latitude")
        longitude = response.json()[0].get("longitude")
        new_mun_coordinates.append([name, latitude, longitude])
    else:
        print("Error:", response.status_code, response.text)
        problems.append(municipality)
    if i %50== 0:
        print(i, " of ", len_todo, " done")

columns = ['Municipality', 'Latitude', 'Longitude']
df_new_mun_coordinates = pd.DataFrame.from_records(new_mun_coordinates, columns=columns)
df_new_mun_coordinates.to_csv("Data/batch5_municipality_coordinates_in_progress")
df_problems = pd.DataFrame(problems)
df_problems.to_csv('Data/final_problems.csv')

0  of  232  done
Error with  Riom-Parsonz
Error with  Oberhofen (AG)
Error with  Tinizong-Rona
Error with  Obererlinsbach
50  of  232  done
Error with  St. Antönien Ascharina
Error with  Nesslau-Krummenau
Error with  Klosters-Serneus
Error with  Suraua
Error with  Schwanden (GL)
Error with  Villaz-Saint-Pierre
Error with  Niedererlinsbach
Error with  Rohr (AG)
Error with  Unterbözberg
Error with  Casti-Wergenstein
Error with  Reckingen-Gluringen
Error with  Tüscherz-Alfermée
Error with  Gettnau
Error with  Thielle-Wavre
Error with  Willisau Stadt
Error with  Bibern (SO)
Error with  Wiesen (GR)
Error with  Willisau Land
Error with  Isorno
Error with  Rüti (GL)
200  of  232  done
Error with  Rapperswil (SG)
Error with  Münster-Geschinen
Error with  Heinrichswil-Winistorf
Error with  Niederried bei Kallnach


In [34]:
df_batch5 = pd.read_csv('Data/batch5_municipality_coordinates_in_progress.csv')
df_until = pd.read_csv('Data/municipality_to_latitude_longitude.csv')
concated = pd.concat([df_batch5, df_until])
concated = concated.drop_duplicates()
concated.to_csv('Data/municipality_to_latitude_longitude.csv')

In [37]:
concated = concated[['Municipality', 'Latitude', 'Longitude']]
concated.to_csv('Data/municipality_to_latitude_longitude.csv', index=False)